In [ ]:
pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 30.1 MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import os, shutil
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
original_dataset_dir = '/content/drive/MyDrive/data'
base_dir = '/content/drive/MyDrive/data'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
conv_base = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3))

16705208/16705208 [==============================] - 0s 0us/step


In [ ]:
conv_base.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 224, 224, 3)  0           ['normalization[0][0

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 1024)              64226304  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 8)                 8200      
                                                                 
Total params: 68,284,075
Trainable params: 68,242,052
Non-trainable params: 42,023
_________________________________________________________________


In [ ]:
print('Это количество тренируемых весов '
            'перед замораживанием conv base:', len(model.trainable_weights))

Это количество тренируемых весов перед замораживанием conv base: 215


In [ ]:
conv_base.trainable = False

In [ ]:
print('Это количество тренируемых весов '
            'после заморозки conv base:', len(model.trainable_weights))

Это количество тренируемых весов после заморозки conv base: 4


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch = 10

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=True)
label_map = (train_generator.class_indices)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=True)

label_map

Found 23219 images belonging to 8 classes.
Found 2585 images belonging to 8 classes.


{'Засветы': 0,
 'Малая часть упаковки видна на фото': 1,
 'Не читабилен текст': 2,
 'Нормальные': 3,
 'Размытые': 4,
 'Фото не в том режиме': 5,
 'Фото полки где 4-5 товара и видно что фоткали не основ товар': 6,
 'Фото ценника': 7}

In [ ]:
train_stop = 3604//batch
test_stop = 401//batch

In [ ]:
from tensorflow.keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
%%time
history = model.fit(train_generator,
                    steps_per_epoch=train_stop,
                    epochs=10)

Epoch 1/10
360/360 [==============================] - 3545s 10s/step - loss: 1.7159 - categorical_accuracy: 0.6194
Epoch 2/10
360/360 [==============================] - 3023s 8s/step - loss: 1.3130 - categorical_accuracy: 0.7091
Epoch 3/10
360/360 [==============================] - 2579s 7s/step - loss: 1.2133 - categorical_accuracy: 0.7458
Epoch 4/10
360/360 [==============================] - 2201s 6s/step - loss: 1.2774 - categorical_accuracy: 0.7642
Epoch 5/10
360/360 [==============================] - 1843s 5s/step - loss: 1.1568 - categorical_accuracy: 0.7866
Epoch 6/10
360/360 [==============================] - 1600s 4s/step - loss: 1.1576 - categorical_accuracy: 0.7975
Epoch 7/10
360/360 [==============================] - 1413s 4s/step - loss: 1.4625 - categorical_accuracy: 0.7869
Epoch 8/10
360/360 [==============================] - 1212s 3s/step - loss: 1.1423 - categorical_accuracy: 0.8097
Epoch 9/10
360/360 [==============================] - 1078s 3s/step - loss: 1.2432 - ca

In [ ]:
test = model.evaluate(test_generator, steps=test_stop)

40/40 [==============================] - 385s 10s/step - loss: 1.5042 - categorical_accuracy: 0.9050


In [ ]:
model.save('/content/drive/MyDrive/dataset/ENetB0.h5')

In [ ]:
import pandas as pd

In [ ]:
df_marks = pd.DataFrame({'x_test': test_generator.filepaths,
     'y_test': test_generator.labels})

x_test = df_marks.drop(['y_test'],axis=1)
y_test = df_marks['y_test']



In [ ]:
pred = model.predict(test_generator)

 93/259 [=========>....................] - ETA: 23:34

KeyboardInterrupt: ignored

In [ ]:
predicted_classes = np.argmax(pred, axis = 1)
print(predicted_classes)

In [ ]:
from sklearn.metrics import classification_report


In [ ]:
report = classification_report(y_test, predicted_classes)
print(report)